In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
wine_data_df = pd.read_csv('./data/vivino_data_241107.csv')
wine_data_df.head() 

,vintage_id,vintage_name,vintage_year,vintage_price,vintage_ratings_average,vintage_ratings_count,vintage_wine_id,vintage_wine_name,vintage_winery,vintage_country,vintage_region,vintage_wine_type_id,acidity,fizziness,intensity,sweetness,tannin,flavor,foods
0,177404982,Carl Loewen 1896 Riesling 2023,2023,82045,5.0,115,1945087,1896 Riesling 2023,Carl Loewen,Germany,Mosel,2,4.61,NaN,3.32,2.05,NaN,tree_fruit,"['Pork', 'Shellfish', 'Spicy food', 'Poultry',..."
1,1510217,Château Haut-Brion Pessac-Léognan (Premier Gra...,1989,3244496,4.8,1538,1152755,Pessac-Léognan (Premier Grand Cru Classé) 1989,Château Haut-Brion,France,Pessac-Léognan,1,4.22,NaN,4.08,1.67,4.23,oak,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul..."
2,2611979,Château Latour Grand Vin Pauillac (Premier Gra...,1982,2759686,4.8,1445,1655970,Grand Vin Pauillac (Premier Grand Cru Classé) ...,Château Latour,France,Pauillac,1,4.12,NaN,4.12,1.68,4.13,oak,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul..."
3,170413977,Domaines Ott Clos Mireille Rosé (Coeur de Grai...,2022,59594,4.8,581,1382222,Clos Mireille Rosé (Coeur de Grain) 2022,Domaines Ott,France,Côtes de Provence,4,3.77,NaN,2.50,1.33,NaN,citrus_fruit,"['Pork', 'Shellfish', 'Vegetarian', 'Poultry']"
4,1688597,Krug Clos du Mesnil 2000,2000,2175423,4.8,347,79635,Clos du Mesnil 2000,Krug,France,Champagne,3,4.54,4.22,3.97,NaN,NaN,non_oak,"['Pork', 'Rich fish (salmon, tuna etc)', 'Shel..."


In [7]:
# 와인 타입 숫자에서 문자열로 변경
# vintage_wine_type_id 
# 1-red, 2-white, 3-sparkling, 4-rose, 7-dessert, 24-fortified
vintage_wine_type_id_dic = {
    1 : 'Red',
    2 : 'White',
    3 : 'Sparkling',
    4 : 'Rose',
    7 : 'Dessert',
    24 : 'Fortified',
    25 : 'Unknown'
}

wine_data_df['vintage_wine_type_id'] = wine_data_df['vintage_wine_type_id'].apply(lambda x: vintage_wine_type_id_dic[x])

# 컬럼명 'vintage_wine_type_id'에서 'vintage_wine_type' 으로 변경
wine_data_df = wine_data_df.rename(columns={'vintage_wine_type_id': 'vintage_wine_type'})

In [21]:
wine_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3675 entries, 0 to 3674
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   vintage_id               3675 non-null   int64 
 1   vintage_name             3675 non-null   object
 2   vintage_year             3675 non-null   object
 3   vintage_price            3675 non-null   int64 
 4   vintage_ratings_average  3675 non-null   int32 
 5   vintage_ratings_count    3675 non-null   int64 
 6   vintage_wine_id          3675 non-null   int64 
 7   vintage_wine_name        3675 non-null   object
 8   vintage_winery           3674 non-null   object
 9   vintage_country          3675 non-null   object
 10  vintage_region           3675 non-null   object
 11  vintage_wine_type        3675 non-null   object
 12  acidity                  3675 non-null   int32 
 13  fizziness                3675 non-null   int32 
 14  intensity                3675 non-null  

In [14]:
# catboost 사용을 위해 속성 컬럼 정수형으로 변경
# 소수점 이하 반올림 처리
# NaN 값은 0 으로 변경
wine_data_df[['acidity', 'fizziness', 'intensity', 'sweetness', 'tannin']] = wine_data_df[['acidity', 'fizziness', 'intensity', 'sweetness', 'tannin', ]].fillna(0).round().astype(int)
wine_data_df.head()

,vintage_id,vintage_name,vintage_year,vintage_price,vintage_ratings_average,vintage_ratings_count,vintage_wine_id,vintage_wine_name,vintage_winery,vintage_country,vintage_region,vintage_wine_type,acidity,fizziness,intensity,sweetness,tannin,flavor,foods
0,177404982,Carl Loewen 1896 Riesling 2023,2023,82045,5,115,1945087,1896 Riesling 2023,Carl Loewen,Germany,Mosel,White,5,0,3,2,0,tree_fruit,"['Pork', 'Shellfish', 'Spicy food', 'Poultry',..."
1,1510217,Château Haut-Brion Pessac-Léognan (Premier Gra...,1989,3244496,5,1538,1152755,Pessac-Léognan (Premier Grand Cru Classé) 1989,Château Haut-Brion,France,Pessac-Léognan,Red,4,0,4,2,4,oak,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul..."
2,2611979,Château Latour Grand Vin Pauillac (Premier Gra...,1982,2759686,5,1445,1655970,Grand Vin Pauillac (Premier Grand Cru Classé) ...,Château Latour,France,Pauillac,Red,4,0,4,2,4,oak,"['Beef', 'Lamb', 'Game (deer, venison)', 'Poul..."
3,170413977,Domaines Ott Clos Mireille Rosé (Coeur de Grai...,2022,59594,5,581,1382222,Clos Mireille Rosé (Coeur de Grain) 2022,Domaines Ott,France,Côtes de Provence,Rose,4,0,2,1,0,citrus_fruit,"['Pork', 'Shellfish', 'Vegetarian', 'Poultry']"
4,1688597,Krug Clos du Mesnil 2000,2000,2175423,5,347,79635,Clos du Mesnil 2000,Krug,France,Champagne,Sparkling,5,4,4,0,0,non_oak,"['Pork', 'Rich fish (salmon, tuna etc)', 'Shel..."


In [22]:
# vintage_winery 와 flavor 컬럼 내 NaN 셀에 None 값 추가
wine_data_df[['vintage_winery','flavor']] = wine_data_df[['vintage_winery', 'flavor']].fillna('None')
wine_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3675 entries, 0 to 3674
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   vintage_id               3675 non-null   int64 
 1   vintage_name             3675 non-null   object
 2   vintage_year             3675 non-null   object
 3   vintage_price            3675 non-null   int64 
 4   vintage_ratings_average  3675 non-null   int32 
 5   vintage_ratings_count    3675 non-null   int64 
 6   vintage_wine_id          3675 non-null   int64 
 7   vintage_wine_name        3675 non-null   object
 8   vintage_winery           3675 non-null   object
 9   vintage_country          3675 non-null   object
 10  vintage_region           3675 non-null   object
 11  vintage_wine_type        3675 non-null   object
 12  acidity                  3675 non-null   int32 
 13  fizziness                3675 non-null   int32 
 14  intensity                3675 non-null  

In [23]:
from sklearn.model_selection import train_test_split
# 데이터 분리 
X = wine_data_df.drop('vintage_ratings_average', axis=1)
y = wine_data_df['vintage_ratings_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2940, 18), (735, 18), (2940,), (735,))

In [24]:
X_train.columns

Index(['vintage_id', 'vintage_name', 'vintage_year', 'vintage_price',
       'vintage_ratings_count', 'vintage_wine_id', 'vintage_wine_name',
       'vintage_winery', 'vintage_country', 'vintage_region',
       'vintage_wine_type', 'acidity', 'fizziness', 'intensity', 'sweetness',
       'tannin', 'flavor', 'foods'],
      dtype='object')

In [25]:
# CatBoost 데이터 준비
# - 범주형 데이터 처리를 위한 Pool 객체 생성
from catboost import Pool

# 카테고리 컬럼명 선언 (연속형 데이터 컬럼은 제외)
cat_features = ['vintage_id', 'vintage_name', 'vintage_year', 'vintage_price',
       'vintage_ratings_count', 'vintage_wine_id', 'vintage_wine_name',
       'vintage_winery', 'vintage_country', 'vintage_region',
       'vintage_wine_type', 'acidity', 'fizziness', 'intensity', 'sweetness',
       'tannin', 'flavor', 'foods'
]


X_train_pool = Pool(X_train, y_train, cat_features=cat_features)
X_test_pool = Pool(X_test, y_test, cat_features=cat_features)

In [26]:
# CatBoostRegressor 회귀 모델 학습
from catboost import CatBoostRegressor

cb_reg = CatBoostRegressor(
    n_estimators=5000, # 반복횟수 (내부 예측기 개수) 
    learning_rate=0.03, # 학습률 (기본값) 
    depth=5, 
    loss_function='RMSE', # 손실함수 (기본값)
    eval_metric='RMSE' # 평가지표 (기본값)
)

# 훈련
cb_reg.fit(X_train_pool, eval_set=X_test_pool, verbose=100) # 100번째마다 출력


0:	learn: 0.1932066	test: 0.1935578	best: 0.1935578 (0)	total: 164ms	remaining: 13m 40s
100:	learn: 0.1646068	test: 0.1650480	best: 0.1650480 (100)	total: 2.19s	remaining: 1m 46s
200:	learn: 0.1588207	test: 0.1633947	best: 0.1628376 (176)	total: 4.08s	remaining: 1m 37s
300:	learn: 0.1555557	test: 0.1643253	best: 0.1628376 (176)	total: 6.02s	remaining: 1m 34s
400:	learn: 0.1524322	test: 0.1645316	best: 0.1628376 (176)	total: 7.99s	remaining: 1m 31s
500:	learn: 0.1498187	test: 0.1644640	best: 0.1628376 (176)	total: 9.99s	remaining: 1m 29s
600:	learn: 0.1475568	test: 0.1645085	best: 0.1628376 (176)	total: 12.1s	remaining: 1m 28s
700:	learn: 0.1458791	test: 0.1651009	best: 0.1628376 (176)	total: 14.1s	remaining: 1m 26s
800:	learn: 0.1440244	test: 0.1649550	best: 0.1628376 (176)	total: 16.2s	remaining: 1m 25s
900:	learn: 0.1415156	test: 0.1647360	best: 0.1628376 (176)	total: 19.2s	remaining: 1m 27s
1000:	learn: 0.1398637	test: 0.1646510	best: 0.1628376 (176)	total: 21.5s	remaining: 1m 25s
1

In [27]:
# 성능에 가장 영향력이 있는 특성은?
pd.DataFrame({
    'column': X_train.columns,
    'importance': cb_reg.feature_importances_
})

,column,importance
0,vintage_id,0.000000
1,vintage_name,0.000000
2,vintage_year,11.030476
3,vintage_price,10.351508
4,vintage_ratings_count,1.305190
5,vintage_wine_id,37.341452
6,vintage_wine_name,0.000000
7,vintage_winery,13.480273
8,vintage_country,4.108299
9,vintage_region,1.528317


In [ ]:
visit_areas = travel_df['VISIT_AREA_NM'].unique()
print(visit_areas.shape)
visit_areas[:5]

In [ ]:
# 사용자 데이터 입력
# 'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
# 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
# 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM',
# 'VISIT_AREA_NM', 'MVMN_NM'
user_input = ['남', 30, 1,2,3,4,5,6,7,8,1,2, '방문지', '자가용']
pred_results = []
for area in visit_areas:
    user_input[-2] = area
    dgstfn_pred = cb_reg.predict(user_input)
    pred_results.append(dgstfn_pred)

pred_results

In [ ]:
# 추천 결과 확인
result_df = pd.DataFrame({
    'VISIT_AREA_NM': visit_areas,
    'DGSTFN_PRED': pred_results,
})
result_df = result_df.sort_values(by='DGSTFN_PRED', ascending=False).head(10)
result_df